In [ ]:
import sys

In [ ]:
cd '/srv/share3/hagrawal9/project/habitat/habitat-api/'

In [ ]:
import gzip
import json
import os
import sys
from typing import Any, Dict, List, Optional, Type

import attr
import cv2
import git
import magnum as mn
import numpy as np

# %matplotlib inline
from matplotlib import pyplot as plt
from pathlib import Path
from PIL import Image


import habitat
import habitat_sim
from habitat.core.registry import registry
from habitat_sim.utils import viz_utils as vut
from habitat.utils.visualizations import maps
from habitat.utils.visualizations.utils import observations_to_image

In [ ]:
from habitat.datasets.rearrangement.rearrangement_dataset import RearrangementDatasetV0
from habitat.tasks.rearrangement.rearrangement_task import RearrangementEpisode
from rearrangement.task.environment import RearrangementRLEnv

In [ ]:
from rearrangement.task.sensors import *

In [ ]:
from habitat_baselines.config.default import get_config

In [ ]:
repo = git.Repo(".", search_parent_directories=True)
dir_path = repo.working_tree_dir
# %cd $dir_path
data_path = os.path.join(dir_path, "data")
output_directory = "data/tutorials/output/"  # @param {type:"string"}
output_path = os.path.join(dir_path, output_directory)

In [ ]:
config = get_config("rearrangement/configs/rearrangement.yaml")

In [9]:
config.defrost()
config.TASK_CONFIG.defrost()
config.TASK_CONFIG.TASK.MEASUREMENTS = ['OBJECT_TO_GOAL_DISTANCE', 'AGENT_TO_OBJECT_DISTANCE', 'TOP_DOWN_MAP']
config.TASK_CONFIG.freeze()
config.freeze()

In [11]:
registry.mapping['sensor']

{'HabitatSimRGBSensor': habitat.sims.habitat_simulator.habitat_simulator.HabitatSimRGBSensor,
 'HabitatSimDepthSensor': habitat.sims.habitat_simulator.habitat_simulator.HabitatSimDepthSensor,
 'HabitatSimSemanticSensor': habitat.sims.habitat_simulator.habitat_simulator.HabitatSimSemanticSensor,
 'PointGoalSensor': habitat.tasks.nav.nav.PointGoalSensor,
 'ImageGoalSensor': habitat.tasks.nav.nav.ImageGoalSensor,
 'PointGoalWithGPSCompassSensor': habitat.tasks.nav.nav.IntegratedPointGoalGPSAndCompassSensor,
 'HeadingSensor': habitat.tasks.nav.nav.HeadingSensor,
 'CompassSensor': habitat.tasks.nav.nav.EpisodicCompassSensor,
 'GPSSensor': habitat.tasks.nav.nav.EpisodicGPSSensor,
 'ProximitySensor': habitat.tasks.nav.nav.ProximitySensor,
 'QuestionSensor': habitat.tasks.eqa.eqa.QuestionSensor,
 'InstructionSensor': habitat.tasks.vln.vln.InstructionSensor,
 'GrippedObjectSensor': habitat.tasks.rearrangement.rearrangement_task.GrippedObjectSensor,
 'AllObjectPositions': habitat.tasks.rearrange

In [12]:
try:
    env.close()
except:
    pass
env = RearrangementRLEnv(config)

2020-10-21 20:02:17,319 Initializing dataset RearrangementDataset-v0
2020-10-21 20:02:17,552 initializing sim RearrangementSim-v0
I1021 20:02:36.322049 54807 simulator.py:168] Loaded navmesh data/scene_datasets/gibson_train_val/Sands.navmesh
I1021 20:02:36.326782 54807 simulator.py:180] Recomputing navmesh for agent's height 0.88 and radius 0.18.
2020-10-21 20:02:36,633 Initializing task RearrangementTask-v0


> /srv/share3/hagrawal9/project/habitat/habitat-api/habitat/core/embodied_task.py(267)_init_entities()
-> entity_cfg = getattr(entities_config, entity_name)


(Pdb)  entity_name


'OBJECT_TO_GOAL_DISTANCE'


(Pdb)  entities_config


...

> /srv/share3/hagrawal9/project/habitat/habitat-api/habitat/core/embodied_task.py(266)_init_entities()
-> pdb.set_trace()


(Pdb)  entities_config[entity_name]


*** KeyError: 'ORACLE_NEXT_OBJECT_SENSOR'
--KeyboardInterrupt--
--KeyboardInterrupt--


(Pdb)  q


BdbQuit: 

In [16]:
config.TASK_CONFIG.TASK.SENSORS

['GRIPPED_OBJECT_SENSOR',
 'ALL_OBJECT_POSITIONS',
 'ALL_OBJECT_GOALS',
 'ORACLE_NEXT_OBJECT_SENSOR',
 'RANDOM_OBJECT_SENSOR',
 'CLOSEST_OBJECT_SENSOR',
 'L2DIST_OBJECT_SENSOR']

In [ ]:
obs = env.reset()
obs_list = []
plt.imshow(obs['rgb'])
# Get closer to the object

# obs = env.step(2)
# obs_list.append(obs)
# metrics = env.get_metrics()
# print_info(obs, metrics)

# Grab the object
# obs = env.step(2)
obs_list.append(obs)
metrics = env.get_metrics()

In [ ]:
obs = env.step(1)
plt.imshow(obs['rgb'])

In [ ]:
obs = env.step(2)
plt.imshow(obs['rgb'])

In [ ]:
obs = env.step(3)
plt.imshow(obs['rgb'])

In [ ]:
obs = env.step(4)
plt.imshow(obs['rgb'])
print(obs['gripped_object_id'])